In [11]:
# --- Standard Library Imports ---
import random
import functools
from collections import defaultdict  # For cluster helper if needed

# --- Third-Party Library Imports ---
import numpy as np
import pandas as pd

# --- Custom Library Imports ---
from models import genetic_ops
from utils.utilities import generate_f1_calendar
from utils.sql import get_table

# --- DEAP Imports ---
try:
    from deap import base, creator, tools, algorithms
except ImportError:
    print("DEAP library not found. Please install it using: pip install deap")
    exit()  # Exit or handle appropriately if DEAP is missing
    
# --- Main GA Execution ---

# Parameters
POPULATION_SIZE = 100
CROSSOVER_PROB = 0.8  # Probability of mating two individuals
MUTATION_PROB = 0.15  # Probability of mutating an individual
NUM_GENERATIONS = 100 # Start with fewer generations for testing
TOURNAMENT_SIZE = 5  # For tournament selection
RANDOM_SEED = 42
SEASON_YEAR = 2026 # For fitness calculation context

# Set random seed
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED) # If numpy random is used internally

# --- Prepare Scenario Data ---
# It must contain 'circuit_name', 'cluster_id', 'start_freq_prob', 'end_freq_prob'
# and any other static data needed by calculate_fitness
circuits_df_scenario = genetic_ops.get_circuits_for_population(season=2025)[['code','cluster_id','first_gp_probability','last_gp_probability']]
circuits_df_scenario.columns = ['circuit_name', 'cluster_id', 'start_freq_prob', 'end_freq_prob']
circuit_list_scenario = circuits_df_scenario['circuit_name'].tolist()
print(f"Optimizing for {len(circuit_list_scenario)} circuits.")
print(f"circuit_list_scenario: {circuit_list_scenario}")


# --- DEAP Setup ---
# Create Fitness and Individual types
# weights=(-1.0,) means we want to minimize the fitness score
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

# Initialize Toolbox
toolbox = base.Toolbox()

# Register functions for creating individuals and population
# CUSTOM INITIAL POPULATION
# Note: Requires generate_initial_population to be fully implemented
initial_pop_list = genetic_ops.generate_initial_population(circuits_df_scenario, POPULATION_SIZE, seed=RANDOM_SEED)
toolbox.register("population_custom", lambda: [creator.Individual(ind) for ind in initial_pop_list])

# Register the genetic operators
# Pass fixed arguments needed by your fitness function
# Ensure 'calculate_fitness' is fully implemented!
toolbox.register("evaluate", genetic_ops.calculate_fitness, 
                 circuits_df=circuits_df_scenario, # Pass needed static data
                 season=SEASON_YEAR, 
                 regression=False, # Set to True once implemented
                 clusters=True, 
                 verbose=False) 
                 
toolbox.register("mate", functools.partial(genetic_ops.order_crossover_deap, toolbox))
toolbox.register("mutate", functools.partial(genetic_ops.swap_mutation_deap, toolbox)) # Your custom swap mutation
toolbox.register("select", tools.selTournament, tournsize=TOURNAMENT_SIZE) # DEAP's tournament

# --- Statistics and Hall of Fame ---
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

hof = tools.HallOfFame(1) # Store only the best individual

# --- Run the GA ---
print("\n--- Starting Genetic Algorithm ---")

population = toolbox.population_custom() 

# Run one of DEAP's algorithms (eaSimple is a basic generational GA)
# Note: eaSimple modifies the population in-place
population, logbook = algorithms.eaSimple(population, toolbox, 
                                         cxpb=CROSSOVER_PROB, 
                                         mutpb=MUTATION_PROB, 
                                         ngen=NUM_GENERATIONS, 
                                         stats=stats, 
                                         halloffame=hof, 
                                         verbose=True) # Prints stats per generation

# --- Results ---
print("\n--- Genetic Algorithm Finished ---")
best_individual = hof[0]
best_fitness = best_individual.fitness.values[0]

print(f"Best Individual Found (Calendar Sequence):")
print(best_individual)
print(f"\nBest Fitness Score Found: {best_fitness}")

# logbook contains stats per generation, useful for plotting convergence
print("\nLogbook:")
print(logbook)


Optimizing for 24 circuits.
circuit_list_scenario: ['BAHSAK', 'AUSMEL', 'CHISHA', 'JAPSUZ', 'SAUJED', 'USAMIA', 'ITAIMO', 'MONMON', 'SPACAT', 'CANMON', 'AUSSPI', 'UKGSIL', 'BELSPA', 'HUNBUD', 'NETZAN', 'ITAMON', 'AZEBAK', 'SINMAR', 'USAAUS', 'MEXMEX', 'BRASAO', 'USALAS', 'QATLUS', 'UAEYAS']

--- Starting Genetic Algorithm ---


c:\Users\massi\AppData\Local\Programs\Python\Python312\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\massi\AppData\Local\Programs\Python\Python312\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	avg   	std  	min    	max   
0  	100   	143576	19555	66176.1	183388
1  	83    	140308	38664.6	65049.9	247231
2  	82    	110367	43173.6	65049.9	228963
3  	83    	80068.1	29046.2	64651  	192784
4  	88    	80936.6	33530  	64106.5	216391
5  	89    	75528.7	25083.6	59274.5	175708
6  	76    	70073.8	20070.3	59265.5	180762
7  	91    	73628.7	29555.9	58331.1	196844
8  	83    	66174.6	21402.4	55944.3	196427
9  	73    	68939.7	30810.5	55944.3	202322
10 	83    	69616.7	29876.9	55863.3	207426
11 	77    	68572.1	30164.2	55863.3	191092
12 	77    	65815.1	23404.6	54505.9	172526
13 	79    	67922.1	29427.4	54424.8	174383
14 	77    	67734.4	24537.6	54377.4	142214
15 	79    	64416.3	23160.4	54377.4	175666
16 	84    	61269.6	19876  	54371.5	181251
17 	77    	65804.5	32604.8	54291.5	186009
18 	83    	63982.5	25451.4	54291.5	176745
19 	83    	64958.8	27547.6	54291.5	174122
20 	80    	63622.5	27503  	54291.5	194222
21 	84    	70736  	32181.1	54291.5	181285
22 	85    	65549.1	32013  	54245  	208367


In [12]:
coords = get_table('fone_geography')

In [13]:
# Filter coords for rows where code_6 is in best_individual
filtered_coords = coords[coords['code_6'].isin(best_individual)]

# Order the DataFrame based on the order of best_individual
ordered_coords = filtered_coords.set_index('code_6').loc[best_individual].reset_index()

# Add the generated calendar to the ordered_coords DataFrame
ordered_coords['calendar'] = generate_f1_calendar(year=2026, n=len(ordered_coords), verbose=False)


In [14]:
import folium
from folium import plugins

# Create a map centered at the midpoint of the coordinates
mid_lat = (ordered_coords['latitude'].min() + ordered_coords['latitude'].max()) / 2
mid_lon = (ordered_coords['longitude'].min() + ordered_coords['longitude'].max()) / 2
map_chart = folium.Map(location=[mid_lat, mid_lon], zoom_start=3)

# Add markers for all coordinates
for _, row in ordered_coords.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']], popup=row['code_6']).add_to(map_chart)

# Add arrows to show the sequence from the first to the last coordinate
for i in range(len(ordered_coords) - 1):
    start = ordered_coords.iloc[i]
    end = ordered_coords.iloc[i + 1]
    arrow = plugins.PolyLineOffset(
        locations=[[start['latitude'], start['longitude']], [end['latitude'], end['longitude']]],
        color='blue',
        weight=5,
        offset=0
    )
    map_chart.add_child(arrow)
    # Add tooltips and mark start and end circuits differently
    for idx, row in ordered_coords.iterrows():
        tooltip = f"Code: {row['code_6']}, Date: {row['calendar']}, Circuit: {row['circuit_x']}, City: {row['city_x']}, Country: {row['country_x']}"
        if idx == 0:  # Start circuit
            folium.Marker(
                location=[row['latitude'], row['longitude']],
                popup=row['code_6'],
                tooltip=tooltip,
                icon=folium.Icon(color='green', icon='play', prefix='fa')
            ).add_to(map_chart)
        elif idx == len(ordered_coords) - 1:  # End circuit
            folium.Marker(
                location=[row['latitude'], row['longitude']],
                popup=row['code_6'],
                tooltip=tooltip,
                icon=folium.Icon(color='red', icon='stop', prefix='fa')
            ).add_to(map_chart)
        else:  # Intermediate circuits
            folium.Marker(
                location=[row['latitude'], row['longitude']],
                popup=row['code_6'],
                tooltip=tooltip
            ).add_to(map_chart)
# Display the map
map_chart

,id,circuit_x,city_x,country_x,continent,latitude,longitude,existing,months_to_avoid,traditional_months,notes,code_6,first_gp_probability,last_gp_probability
0,1,Silverstone,Silverstone,United Kingdom,Europe,52.0786,-1.0169,1,"[11, 12, 1, 2]",[7],UK winters are cold/rainy; race usually mid-se...,UKGSIL,0.000000,0.000000
1,2,Budapest,Mogyoród,Hungary,Europe,47.5789,19.2486,1,[],"[7, 8]","Good summer weather, scheduled pre-summer break",HUNBUD,0.000000,0.000000
2,3,Monza,Monza,Italy,Europe,45.6156,9.2811,1,"[11, 12, 1, 2]",[9],Held in September post-summer break,ITAMON,0.000000,0.000000
3,4,São Paulo,São Paulo,Brazil,South America,-23.7036,-46.6997,1,[],[11],"Rainy season possible, usually a late-season race",BRASAO,0.000000,0.269231
4,5,Sakhir,Sakhir,Bahrain,Asia,26.0325,50.5106,1,"[6, 7, 8, 9]",[3],Avoid summer heat; often season opener,BAHSAK,0.230769,0.000000


In [4]:
best_individual

['JAPSUZ',
 'USALAS',
 'USAAUS',
 'MEXMEX',
 'USAMIA',
 'CANMON',
 'BRASAO',
 'SAUJED',
 'QATLUS',
 'UAEYAS',
 'BAHSAK',
 'AZEBAK',
 'AUSSPI',
 'BELSPA',
 'NETZAN',
 'UKGSIL',
 'SPACAT',
 'ITAIMO',
 'MONMON',
 'ITAMON',
 'HUNBUD',
 'CHISHA',
 'SINMAR',
 'AUSMEL']